<a href="https://colab.research.google.com/github/felipenjonas/lab_bdag/blob/main/RendaAgricola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
ds = pd.read_csv(url)
ds.head()


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


Separando o dataset em 2 variáveis, uma chamada "Previsores" e a outra "classe"

Sintaxe do ILOC é [linhas,colunas/range de colunas], neste caso, eu quero todas as linhas, por isso uso ":" (2 pontos = todos) e depois quero a coluna na posição 0 até posição 14, então "0:14"

então no final temos: 
 iloc[:,0:14]

In [2]:
previsores = ds.iloc[:,0:14].values
classe = ds.iloc[:,14].values

print(classe)

[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' >50K']


# Tratamento dos dados
Agora, vamos usar o método LabelEncoder da Lib Sklearn.processing para fazer a substituição dos dados das colunas categóricas por números.

In [3]:
from sklearn.preprocessing import LabelEncoder
label_previsores = LabelEncoder()

# Previsores agora é uma matriz, então podemos continuar com a mesma lógica de linhasxcolunas

previsores[:,1] = label_previsores.fit_transform(previsores[:,1])
previsores[:,3] = label_previsores.fit_transform(previsores[:,3])
previsores[:,5] = label_previsores.fit_transform(previsores[:,5])
previsores[:,6] = label_previsores.fit_transform(previsores[:,6])
previsores[:,7] = label_previsores.fit_transform(previsores[:,7])
previsores[:,8] = label_previsores.fit_transform(previsores[:,8])
previsores[:,9] = label_previsores.fit_transform(previsores[:,9])
previsores[:,13] = label_previsores.fit_transform(previsores[:,13])

label_classe = LabelEncoder()

# 1 coluna apenas
classe = label_classe.fit_transform(classe)
classe

array([0, 0, 0, ..., 0, 0, 1])

# Boas práticas: Escalonamento de dados
Standard Scaler

In [4]:
# Padronizando os dados da variável previsores com o Standard Scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

previsores 


array([[ 0.83709708,  1.46388339, -1.0087417 , ..., -0.21666301,
        -2.22212013,  0.29157343],
       [-0.04264043,  0.09012148,  0.24504633, ..., -0.21666301,
        -0.03542999,  0.29157343],
       [ 1.05703146,  0.09012148,  0.42576955, ..., -0.21666301,
        -0.03542999,  0.29157343],
       ...,
       [ 1.42358875,  0.09012148, -0.3588108 , ..., -0.21666301,
        -0.03542999,  0.29157343],
       [-1.21562378,  0.09012148,  0.11092744, ..., -0.21666301,
        -1.65520046,  0.29157343],
       [ 0.98372   ,  0.77700244,  0.92986178, ..., -0.21666301,
        -0.03542999,  0.29157343]])

# Treinamento e teste

[Reamostragem conceitos](http://cursos.leg.ufpr.br/ML4all/apoio/reamostragem.html#__m%C3%A9todos_de_reamostragem__)

In [5]:
from sklearn.model_selection import train_test_split
# Previsores é o X e a classe é o Y, no contexto de outros exemplos 
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores,classe,test_size = 0.5, random_state=0)



# Classificação

In [6]:
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()

classificador.fit(previsores_treinamento, classe_treinamento)
previsores = classificador.predict(previsores_teste)


from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score
# Matriz de confusão
matriz = confusion_matrix(classe_teste, previsores)
print("Matriz de confusão: \n", matriz)


# Acuracia
acuracia = accuracy_score(classe_teste,previsores)
print("\nAcurácia: ", acuracia)

# Recall (Sensibilidade)
recall = average_precision_score(classe_teste,previsores)
print("\nRecall: ", recall)

# Recall manual:
recall2 = 5898 / 6193
print("\nRecall Manual: ",recall2)

# Precisao
precisao = 5898 / (5898+1307)
print("\nPrecisão: ", precisao)

Matriz de confusão: 
 [[11781   591]
 [ 2590  1318]]

Acurácia:  0.8046068796068796

Recall:  0.39193774300214035

Recall Manual:  0.9523655740352011

Precisão:  0.8185981956974323


# RandomForest


In [7]:
import numpy as np
# APLICANDO Random Forest
from sklearn.ensemble import RandomForestRegressor
# Ja importamos o train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
# n_jobs=-1 (usa todos os threads da CPU)
# arveres = RandomForestRegressor( random_state=0, n_jobs=-1)
# Aplicando mais 100 árvores (padrão), o ideal é criar o máximo que o hardware permite para obter cada vez meno overfitting
arveres = RandomForestRegressor(n_estimators=1000, random_state=0, n_jobs=-1)

arveres.fit(previsores_treinamento,classe_treinamento)

previsao1 = arveres.predict(previsores_teste)

# Captura de erro:
# Este é valor do erro da nossa árvore.
np.sqrt(mean_squared_error(classe_teste, previsao1))


0.3195516359438612

Erro Default: 0.32034415998326055 <br>
Erro com n_estimators=200: 0.3200829058950066 <br>
Erro com n_estimators=500: 0.3196850473263169 <br>
Erro com n_estimators=1000: 0.3195516359438612 <br>

A diferença entre usar uma árvore e uma RandomForest é justamente a aleatoriedade aplicada na feature e nas linhas/amostras. <br> Será criado uma amostra de reposição (bootstrap sampling ou bagging). Não é uma amostra fiél da realidade, mas sim uma maneira sintética de gerar mais amostras. 

Além disso, também é usado o conceito de Subspace Sampling. <br> A ideia é usar 2 features aleatorias de cada amostra entre as N features.